In [ ]:
import os
import io
import urllib.request as request
import numpy as np
import pandas as pd
import tensorflow as tf

#Khai báo params
IRIS_TRAIN = 'iris_training.csv'
IRIS_TRAIN_PATH = 'https://raw.githubusercontent.com/phamdinhkhanh/Tensorflow/master/iris_training.csv'
IRIS_TEST = 'iris_testing.csv'
IRIS_TEST_PATH  = 'https://raw.githubusercontent.com/phamdinhkhanh/Tensorflow/master/iris_testing.csv'
COLUMNS = ['SepWid', 'SepLen', 'PenWid', 'PenLen', 'Species']
BATCH_SIZE = 100
N_STEPS = 2000
LEARNING_RATE = 0.1

def get_data(url, filename, is_get_train = True):
    if not os.path.exists(filename):
        raw = request.urlopen(url).read().decode('utf-8')
        with io.open(filename, 'w') as f:
            f.write(raw)
            
    data = pd.read_csv(filename, header = 0, names = COLUMNS, encoding = 'utf-8')
    features, labels = data, data.pop('Species')
    
    #Tạo Class Dataset trong tensorflow
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if is_get_train: 
        dataset = dataset.shuffle(1000).repeat().batch(batch_size = BATCH_SIZE)
        return dataset.make_one_shot_iterator().get_next()
    else:
        return dataset.batch(batch_size = BATCH_SIZE)

get_data(IRIS_TRAIN_PATH, IRIS_TRAIN, is_get_train = True)
get_data(IRIS_TEST_PATH, IRIS_TEST,  is_get_train = False)

with tf.Session() as sess:
    test = get_data(IRIS_TEST_PATH, IRIS_TEST, is_get_train = False)
    print(sess.run(test.make_one_shot_iterator().get_next()))
      

def my_model_dropout(features, labels, mode, params):
    # 1. Huấn luyện mô hình
    # Xây dựng mạng nơ ron
    # Khởi tạo input_layer. Hàm input_layer sẽ map dữ liệu đầu vào là features với các estimators thông qua params['features_columns']
    nn = tf.feature_column.input_layer(features, params['feature_columns'])
    # Dropout ở input nên có giá trị là 0.7-0.8
    nn = tf.nn.dropout(nn, keep_prob = 0.8)
    # Xây dựng các hidden layer tiếp theo
    for n_units in params['hidden']:
        nn = tf.layers.dense(nn, n_units, activation = tf.nn.relu)
        # Thêm một layer dropout để giảm overfiting
#         nn = tf.nn.dropout(nn, keep_prob = params['dropout'])
    # Hàm logits dự báo xác xuất các classes (chính là output layer)
    logits = tf.layers.dense(nn, params['n_classes'], activation = tf.nn.softmax)
    
    # Hàm loss function
    loss = tf.losses.sparse_softmax_cross_entropy(
        labels = labels, 
        logits = logits)
    # Hàm tối ưu hóa kiểm soát thuật toán gradient descent:
    # optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE)
    optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE)
    # Hàm kích hoạt quá trình training mô hình:
    train_op = optimizer.minimize(
        loss = loss, 
        global_step = tf.train.get_global_step())
    # Estimator trả về 
    if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(
            mode = mode, 
            loss = loss, 
            train_op = train_op)
    
    #---------------------------------------------------------------------------------
    # 2. Đánh giá mô hình
    elif mode == tf.estimator.ModeKeys.EVAL:
        # Lớp được dự báo 
        prediction_classes = tf.argmax(logits, 1)
        # Mức độ chính xác 
        accuracy = tf.metrics.accuracy(
            labels = labels,
            predictions = prediction_classes
        )
        # Estimator trả về
        return tf.estimator.EstimatorSpec(
            mode = tf.estimator.ModeKeys.EVAL,
            loss = loss,
            eval_metric_ops = {'accuracy': accuracy}
        )
    
classifier2 = tf.estimator.Estimator(
    model_fn = my_model_dropout,
    params = {
        'feature_columns': my_features, # List tên các feature sử dụng để map dữ liệu từ Dataset với các estimator
        'hidden':[10, 20, 10], # Số đơn vị mỗi layer
        'n_classes': 3, # Số lượng nhóm cần phân loại
        'dropout': 0.5
    }
)

classifier2.train(
    input_fn = lambda:get_data(IRIS_TRAIN_PATH, IRIS_TRAIN),
    steps = N_STEPS
)


classifier2.evaluate(
    input_fn = lambda:get_data(IRIS_TRAIN_PATH, IRIS_TRAIN, is_get_train = False)
)

classifier2.evaluate(
    input_fn = lambda:get_data(IRIS_TEST_PATH, IRIS_TEST, is_get_train = False)
)